# Notebook 2.4.5: The Coffee Cooling Problem

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

We've done a lot of guess-and-adjust work to find various parameters: growth rates, shift constants, heat transfer coefficients and so on.  

<br>

It's time we started using the power of Python to solve some of those problems more efficiently.

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/python.PNG width = 400>
</center>

Run this cell to pull in the code from the previous notebook:

In [ ]:
import numpy as np
import pandas as pd
def change_func(t, T, system):
    r, T_env, dt = system['r'], system['T_env'], system['dt']
    deltaT = -r * (T - T_env) * dt
    return deltaT

def run_simulation(system, change_func):

    t_array = np.arange(0, system['t_end']+1, system['dt'])
    n = len(t_array)

    results = pd.Series(index=t_array,dtype=object)
    results.index.name = 'Time (m)'
    results.name = 'Temp (C)'
    results.iloc[0] = system['T_init']

    for i in range(n-1):
        t = t_array[i]
        T = results.iloc[i]
        results.iloc[i+1] = T + change_func(t, T, system)

    system['T_final'] = results.iloc[-1]
    return results

## Using root_scalar

We want to find a value of `r` that produces the correct final temperature of 70 °C.   To do that, we'll use a *root-finding algorithm* called `root_scalar`.

### Finding the roots of an equation

The SciPy library, which is designed for scientific computing, provides a function that finds the roots of non-linear equations. What does that mean?  Consider this quadratic equation:

<br>

$$f(x) = (x - 1)(x - 2)(x - 3)$$

<br>

A *root* is a value of $x$ that makes $f(x)=0$. In this case, we can see that if $x=1$, the first term is 0; if $x=2$, the second term is 0; and if $x=3$, the third term is 0.  So the roots of this equation are 1, 2, and 3.

<br>

Let's use this algorithm to solve this same problem.  First, we have to write a function that evaluates $f$:

In [ ]:
def quad_func(x):
    return (x-1) * (x-2) * (x-3)

Now we call `root_scalar` like this:

In [ ]:
import scipy.optimize as spo
# Find the roots of quad_func
returned_object = spo.root_scalar(quad_func, bracket=[1.5, 2.5])
returned_object


The object returned by the function is an object that contains several variables, including the Boolean value `converged`, which is `True` if the  search converged successfully on a root, and `root`, which is the root that was found.






### How root_scalar works

But notice that algorithm only finds one root.  Let's look and see why. The second
argument in `root_scalar` is an interval that contains or *brackets* a root. We have to tell the function where to look for a root, and it will only find a single root in that interval.

<br>

If we provide a different interval, we find a different root.

In [ ]:
returned_object = spo.root_scalar(quad_func, bracket=[2.5, 3.5])
returned_object

If the interval doesn't contain a root, or if it contains more than one root, you'll get an error.



To understand why, we need to know something about how the function works.
 According to the documentation, `root_scalar` uses "a combination of bisection, secant, and inverse quadratic interpolation methods."

<br>

That's a mouthful!  Suppose we're trying to find a root of a function of one variable, $f(x)$, and assume we have evaluated the function at two places, $x_1$ and $x_2$, and found that the results have opposite signs. Specifically, assume $f(x_1) > 0$ and $f(x_2) < 0$, as shown in the following diagram:

<br>

<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/secant.PNG width = 300>

If $f$ is a continuous function, there must be at least one root in this
interval: the line has to cross the x-axis at some point. In this case we would say that $x_1$ and $x_2$ *bracket* a
root.  If this were all you knew about $f$, where would you go looking for a
root?

* If you said "halfway between $x_1$ and $x_2$," congratulations:
You just invented a numerical method called *bisection*, you clever dog!

* If you said, "I would connect the dots with a straight line and compute
the zero of the line," you just invented the *secant
method*!  You're on a roll!

* And if you said, "I would evaluate $f$ at a third point, find the
parabola that passes through all three points, and compute the zeros of
the parabola," you just invented *inverse quadratic
interpolation*! (If you said this, you may need to be in a higher level class 😀)

That's most of how `root_scalar` works!

---

<br>

🟨 🟨 Active Reading








Now we can explain why we get an error. Run the code below, which sets the bracket between 1.5 and 3.5:

In [ ]:
returned_object = spo.root_scalar(quad_func, bracket=[1.5, 3.5])
returned_object

Read the error message carefully, and compare the error to how `root_scalar` works.  What do you think `f(a) and f(b) must have different signs` means?  Why does the function not work in this situation?  Try some different bracket values to test your theory.

<br>

<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/root_scalar_plot.PNG width = 400>

✅ ✅ Put your answer here

---

### Using root_scalar to find the heat transfer coefficient $r$

Remember that $r$ is the heat transfer coefficient in Newton's Law of Cooling.  It is a complex value determined by air movement, insulation, and surface area, among other things.  What we want is the value of $r$ that yields a final temperature of
70 °C, because that is the temperature we found experimentally.

<br>

To use `root_scalar`, we need a function that takes $r$ as an argument and returns the *difference* between the final temperature and the goal.  In other words, it returns the *error* between our simulation and the expected results:

In [ ]:
def error_func(r, system):
    system['r'] = r
    results = run_simulation(system, change_func)
    return system['T_final'] - 70

With the right value of `r`, the error is 0: this is the root of the function.  We can test `error_func` like this, using the initial guess `r=0.01`.  We should expect to get 2.3 °C, since our result in the previous notebook was 72.3 °C:


In [ ]:
# Make a system
T_init = 90; T_env = 22
volume = 300;    r = 0.01
t_end = 30;  dt = 1.0
coffee = dict(T_init=T_init, T_env=T_env,
              volume=volume, r=r, t_end=t_end, dt=dt)

# Call our error function
error_func(0.01, coffee)

It works!  But because $r = 0.1$ is too small, not enough heat is being transferred out of our model coffee cup.

<br>

To provide valid brackets for `root_scalar`, we need a value for $r$ that is a produces an error that is a different sign than 2.3 °C.  So we want to find a value that transfers *too much* heat.  Let's try `r = 0.02`:

In [ ]:
error_func(0.02, coffee)

With `r=0.02`, the error is  about -11°C, which means that the final temperature is too low. So we know that the correct value must be in between 0.01 and 0.02.  This gives us two values for the bracket argument in `root_scalar`:

In [ ]:
root_obj = spo.root_scalar(error_func, coffee, bracket=[0.01, 0.02])

Notice that we now have 3 arguments when we call `root_scalar`.  The first argument is the error function.
The second argument is the system object, which `root_scalar` passes as an argument to `error_func`.
The third argument is an interval that brackets the root.Here's the root we found.

In [ ]:
r_coffee = root_obj.root
r_coffee

In this example, `r_coffee` turns out to be about `0.0115`, in units of min$^{-1}$ (inverse minutes, or more colloquially, 'per minute').
We can confirm that this value is correct by setting `r` to the root we found and running the simulation.

In [ ]:
coffee['r'] = root_obj.root
run_simulation(coffee, change_func)
coffee['T_final']

The final temperature is very close to 70 °C.

<br>

---

Using an algorithm like `root_scalar` can be tricky to get your head around.  But here's one more way to think about it.  `error_func` literally creates a mathematical function: it provides the error in temperature as a function of the $r$ value that we use.   Here is a plot of that function:

<br>

<center>
<img src = https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Images_and_Data/Images/2_4/coffee_error.PNG width = 400>
</center>

Notice some key points:

<br>

- at $r = 0.01$, the error is 2.3 °C (just as we found)
- at $r = 0.02$, the error is about -11 °C
-most importantly, the plot crosses zero at about $r=0.0115$.  That is, the root of the function is approximately 0.0115.

Seeing this plot may help you understand that we were just finding the root of a function, in the same way we found the roots of the quadratic function at the beginning of this notebook.

## Exercises


---

<br>

🟨 🟨

### Exercise 1

Write an error function that simulates the temperature of the milk and returns the difference between the final temperature and 20 °C.  Use it to find the best value of `r` for the milk.  Use the coffee code above as your direct model.

In [ ]:
# Define error function for milk
def error_func_milk(r, system):


In [ ]:
# Test your function with a guessed value of r = 0.13
res1 = error_func_milk(0.13,milk)
res1

In [ ]:
# Use root_scalar and your error_func_milk to find the r value for milk
